<a href="https://colab.research.google.com/github/RochaErik/AlgorithmComparison/blob/main/AlgorithmComparison3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# **Evaluating algorithms with hyperparameter tuning**

---

In [1]:
!pip install catboost
!pip install lightgbm
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [3]:
!pip install hyperopt

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# **Wine Dataset**

In [8]:
wine_df = pd.read_csv('E:\Cursos\MestradoCienciaComputação\Seminario\Datasets\Wine\wine.data', header=None)

In [9]:
X = wine_df.iloc[:, 1:]
y = wine_df.iloc[:, 0]

In [10]:
le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
import hyperopt.pyll
from hyperopt.pyll import scope

best_hyperparams = {
    'AdaBoost': {},
    'LightGBM': {}
}

# Define the hyperparameter search space for each algorithm

def optimize_adaboost(params):
    estimator_params = params['estimator']
    estimator = DecisionTreeClassifier(**estimator_params)

    clf = AdaBoostClassifier(estimator=estimator, n_estimators=params['n_estimators'], learning_rate=params['learning_rate'], random_state=params['random_state'])
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return -accuracy_score(y_test, y_pred)

def optimize_gradientboost(params):
    clf = GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return -accuracy_score(y_test, y_pred)

def optimize_catboost(params):
    clf = CatBoostClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return -accuracy_score(y_test, y_pred)

def optimize_lightgbm(params):
    clf = LGBMClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return -accuracy_score(y_test, y_pred)

def optimize_xgboost(params):
    clf = XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return -accuracy_score(y_test, y_pred)

# Define the hyperparameter search space for each algorithm

space_adaboost = {
    'n_estimators': 1 + scope.int(hp.quniform('n_estimators', 5, 1500, 50)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.1)),
    'estimator': {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 6, 1)),  # Decision tree depth
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 8, 2)),  # Min samples required to split
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 5, 1)),  # Min samples required in a leaf node
        'max_features': hp.choice('max_features', [None, 'sqrt', 'log2']),
    },
    'random_state': 42
}

# space_gradientboost = {
#     'n_estimators': hp.choice('n_estimators', range(50, 200)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'max_depth': hp.choice('max_depth', range(1, 11)),
# }

# space_catboost = {
#     'iterations': hp.choice('iterations', range(50, 200)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'silent': True
# }

class_weight_choices = ['balanced']
boosting_type_choices = ['gbdt', 'dart', 'goss']
space_lightgbm = {
    'class_weight': hp.choice('class_weight', class_weight_choices),                                              
    'boosting_type': hp.choice('boosting_type', boosting_type_choices),
    'num_leaves': scope.int(hp.quniform('num_leaves', 30, 100, 5)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.1)),
    'min_child_samples': scope.int(hp.quniform('min_child_samples', 20, 200, 10)),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'verbosity': -1,
    'random_state': 42
}

# space_xgboost = {
#     'n_estimators': hp.choice('n_estimators', range(50, 200)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'max_depth': hp.choice('max_depth', range(1, 11)),
# }

# Define optimization functions and algorithm names
optimizers = [
    (optimize_adaboost, space_adaboost, 'AdaBoost'),
    # (optimize_gradientboost, space_gradientboost, 'Gradient Boosting'),
    # (optimize_catboost, space_catboost, 'CatBoost'),
    (optimize_lightgbm, space_lightgbm, 'LightGBM'),
    # (optimize_xgboost, space_xgboost, 'XGBoost'),
]

# Perform hyperparameter tuning for each algorithm
for optimize_fn, space, algorithm_name in optimizers:
    if algorithm_name == 'AdaBoost':
        trials = Trials()
        best = fmin(fn=optimize_fn, space=space, algo=tpe.suggest, max_evals=50, trials=trials)
        
        # Store the best AdaBoost hyperparameters
        best_hyperparams[algorithm_name] = {
            'n_estimators': best['n_estimators'],
            'learning_rate': best['learning_rate'],
            'max_depth': best['max_depth'],
            'max_features': best['max_features'],
            'min_samples_leaf': best['min_samples_leaf'],
            'min_samples_split': best['min_samples_split'],
            'random_state': 42
        }

        print(f"Best hyperparameters for {algorithm_name}:")
        print(best_hyperparams[algorithm_name])
    
    
    if algorithm_name == 'LightGBM':
        trials = Trials()
        best = fmin(fn=optimize_fn, space=space, algo=tpe.suggest, max_evals=50, trials=trials)
        
        # Map the choice labels
        class_weight_label = class_weight_choices[best['class_weight']]
        boosting_type_label = boosting_type_choices[best['boosting_type']]

        # Store the best LightGBM hyperparameters
        best_hyperparams[algorithm_name] = {
            'class_weight': class_weight_label,
            'boosting_type': boosting_type_label,
            'num_leaves': int(best['num_leaves']),
            'learning_rate': best['learning_rate'],
            'min_child_samples': int(best['min_child_samples']),
            'reg_alpha': best['reg_alpha'],
            'reg_lambda': best['reg_lambda'],
            'colsample_by_tree': best['colsample_by_tree'],
            'verbosity': -1,
            'random_state': 42
        }

        print(f"Best hyperparameters for {algorithm_name}:")
        print(best_hyperparams[algorithm_name])









100%|██████████| 50/50 [00:29<00:00,  1.71trial/s, best loss: -1.0]              
Best hyperparameters for AdaBoost:
{'n_estimators': 100.0, 'learning_rate': 0.08634998092687633, 'max_depth': 4.0, 'max_features': 1, 'min_samples_leaf': 4.0, 'min_samples_split': 2.0, 'random_state': 42}
100%|██████████| 50/50 [00:01<00:00, 41.48trial/s, best loss: -1.0]               
Best hyperparameters for LightGBM:
{'class_weight': 'balanced', 'boosting_type': 'gbdt', 'num_leaves': 100, 'learning_rate': 0.07027949493727456, 'min_child_samples': 20, 'reg_alpha': 0.6969013278160577, 'reg_lambda': 0.1448149279595581, 'colsample_by_tree': 0.8929428154866978, 'verbosity': -1, 'random_state': 42}


In [48]:
best_hyperparams['AdaBoost']

{'n_estimators': 100.0,
 'learning_rate': 0.08634998092687633,
 'max_depth': 4.0,
 'max_features': 1,
 'min_samples_leaf': 4.0,
 'min_samples_split': 2.0,
 'random_state': 42}

In [49]:
best_hyperparams['LightGBM']

{'class_weight': 'balanced',
 'boosting_type': 'gbdt',
 'num_leaves': 100,
 'learning_rate': 0.07027949493727456,
 'min_child_samples': 20,
 'reg_alpha': 0.6969013278160577,
 'reg_lambda': 0.1448149279595581,
 'colsample_by_tree': 0.8929428154866978,
 'verbosity': -1,
 'random_state': 42}

In [14]:
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=42)

In [20]:
names = ['AdaBoost', 'LightGBM']

In [51]:
wine_scores = []
wine_scores_mean = []
wine_scores_std = []
model_names = []

for algorithm_name in names:
    if algorithm_name == 'AdaBoost':
        base_estimator = DecisionTreeClassifier(max_depth=int(best_hyperparams[algorithm_name]['max_depth']),
                                                max_features=best_hyperparams[algorithm_name]['max_features'],
                                                min_samples_leaf=int(best_hyperparams[algorithm_name]['min_samples_leaf']),
                                                min_samples_split=int(best_hyperparams[algorithm_name]['min_samples_split']))

        clf = AdaBoostClassifier(estimator=base_estimator, 
                                n_estimators=int(best_hyperparams[algorithm_name]['n_estimators']), 
                                learning_rate=best_hyperparams[algorithm_name]['learning_rate'],
                                random_state=42)              
        
    if algorithm_name == 'LightGBM':
        clf = LGBMClassifier(boosting_type=best_hyperparams[algorithm_name]['boosting_type'], 
                            class_weight=best_hyperparams[algorithm_name]['class_weight'], 
                            colsample_by_tree=best_hyperparams[algorithm_name]['colsample_by_tree'],
                            learning_rate=best_hyperparams[algorithm_name]['learning_rate'],
                            min_child_samples=best_hyperparams[algorithm_name]['min_child_samples'],
                            num_leaves=best_hyperparams[algorithm_name]['num_leaves'],
                            reg_alpha=best_hyperparams[algorithm_name]['reg_alpha'],
                            reg_lambda=best_hyperparams[algorithm_name]['reg_lambda'],
                            verbosity=-1,
                            random_state=42)
        
    results = cross_val_score(clf, X, y, cv=rskf)
    wine_scores.append(results)
    wine_scores_mean.append(results.mean()*100)
    wine_scores_std.append(results.std()*100)
    model_names.append(algorithm_name)
    print(f'--------- {algorithm_name} on Wine Dataset ---------')
    print(results)
    print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
    print('------------------------------')    
                        




#   results = cross_val_score(clf, X, y, cv=rskf)
#   wine_scores.append(results)
#   wine_scores_mean.append(results.mean()*100)
#   wine_scores_std.append(results.std()*100)
#   model_names.append(name)
#   print(f'--------- {name} on Wine Dataset ---------')
#   print(results)
#   print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
#   print('------------------------------')



--------- AdaBoost on Wine Dataset ---------
[0.94444444 0.94444444 1.         1.         0.94444444 1.
 1.         0.83333333 1.         1.         1.         1.
 0.94444444 1.         1.         0.94444444 1.         0.94444444
 1.         0.94117647 1.         1.         0.88888889 1.
 1.         1.         1.         0.94444444 0.94117647 1.
 0.94444444 1.         1.         0.83333333 1.         1.
 1.         1.         1.         0.94117647 0.94444444 1.
 1.         0.94444444 1.         1.         1.         1.
 0.94117647 1.         1.         0.94444444 1.         0.94444444
 1.         1.         0.94444444 0.94444444 1.         0.94117647
 1.         1.         0.94444444 1.         1.         1.
 0.94444444 0.94444444 1.         1.         1.         1.
 1.         0.94444444 1.         1.         1.         1.
 0.94117647 0.94117647 1.         0.94444444 1.         0.94444444
 1.         0.94444444 1.         1.         0.94117647 1.
 1.         0.94444444 1.         0.94